# About Florida Insurance Analysis

Book combines historical publicly reported data from the FL department of insurance to evaluate trends in market share, claims frequency/severity, and hurricane losses.

Data downloaded from https://apps.fldfs.com/QSRNG/Reports/ReportCriteria.aspx on 04 and 05 April 2025.

Environment AKStandard (Python 3.11.9)

# Imports

In [30]:
import pandas as pd # data ingestion, manipulation
from os import listdir # importing raw data
import time # I just want to very briefly look at data and have it print for a second to glance at it
from IPython.display import clear_output # thanks to ChatGPT - lets me clear printed data

# Data Extraction/Ingestion

Outline/Plan

function - import, remove grand total\
function - check/remove blank columns\
compile to a single df\
\
future: move to AW S3 bucket, SQL access

In [3]:
# listdir('./data/raw')[-1]

In [4]:
info = pd.read_excel('./data/raw/2009Q1.xlsx', skiprows = 14)
print(f"The imported shape is: {info.shape}\nThe last named insurers are:\n{info.iloc[-2:,1]}\n")
grand_total_row = info.shape[0] - 1
info.drop(grand_total_row, axis=0, inplace=True)
print(f"The updated shape is: {info.shape}\nThe last named insurers are:\n{info.iloc[-2:,1]}")

The imported shape is: (214, 65)
The last named insurers are:
212    TESLA PROPERTY & CASUALTY, INC.
213                                NaN
Name: Insurer Name, dtype: object

The updated shape is: (213, 65)
The last named insurers are:
211    STATE NATIONAL INSURANCE COMPANY INC.
212          TESLA PROPERTY & CASUALTY, INC.
Name: Insurer Name, dtype: object


In [21]:
len(info.columns)

65

In [50]:
test = 'this is a test string thingy line'
if "hi" in test:
    print('hi')

hi


In [ ]:
counter = 0
for x in info.columns:
    counter += 1
    clear_output(wait = True)
    print(f"Iteration: {counter} Column: {x}\n", info[x])
    time.sleep(0.2)

Iteration 65 Column Unnamed: 64
 0      0%
1      0%
2      0%
3      0%
4      0%
       ..
208    0%
209    0%
210    0%
211    0%
212    0%
Name: Unnamed: 64, Length: 213, dtype: object


In [48]:
info.columns[4]

'Unnamed: 4'

In [53]:
for x in info.columns:
    if "Unnamed" not in x:
        print(x)
    else:
        pass

Rank
Insurer Name
Policies In Force
Total $ value of exposure for policies in force that include wind coverage
Number of policies canceled
Number of policies nonrenewed
Number of policies canceled due to hurricane risk
Number of policies nonrenewed due to hurricane risk
Number of new policies written
Policies in force that exclude wind coverage
Total premiums written
Policies in force that include wind coverage
Total $ value of exposure for policies in force that exclude wind coverage
Direct premium written for policies in force that include wind coverage
Direct premium written for policies in force that exclude wind coverage
Number of policies transferred to other insurers
Number of policies received from other insurers
Claims Opened
Claims Closed
Claims Pending
Claims Alternative Dispute
Claims with Mediation
Claims with Arbitration
Claims with Appraisal
Claims with Sink Hole Eval
Claims with Settlement Conf
Claims with Other


In [32]:
import time

items = ["Item 1", "Item 2", "Item 3", "Item 4"]

for item in items:
    clear_output(wait = True)
    print(item)
    time.sleep(1)

# Optionally move to a new line after the loop
print()


Item 4



In [ ]:
# chatty G getting wild

from IPython.display import display, clear_output
import ipywidgets as widgets

items = ["Apple", "Banana", "Cherry", "Date"]
saved_items = []
index = 0  # Keeps track of which item we're on

# Buttons
save_button = widgets.Button(description="Save")
skip_button = widgets.Button(description="Skip")
output = widgets.Output()

def handle_click(button):
    global index
    if button == save_button:
        saved_items.append(items[index])
    
    index += 1
    if index < len(items):
        show_item()
    else:
        clear_output()
        print("Done! Saved items:", saved_items)

def show_item():
    clear_output(wait=True)
    print("Current item:", items[index])
    display(save_button, skip_button)

save_button.on_click(handle_click)
skip_button.on_click(handle_click)

# Start it off
show_item()


ModuleNotFoundError: No module named 'ipywidgets'